In [27]:
import os
import glob
import argparse 
import subprocess as sub
import shlex 
from subprocess import PIPE, Popen
import pandas as pd
import logging
import re
import sys
import datetime

In [28]:
def getting_bnum_tnum_list(bnum_tnum_csv): 
    ########### Get bnum_tnum list: 
    savedir = os.getcwd()
    os.chdir(csv_dir)
    bnum_tnum_df = pd.read_csv(bnum_tnum_csv, header = None)
    bnum_tnum_df.columns = ['bnum', 'tnum', 'DUMMY']
    os.chdir(savedir)
    return bnum_tnum_df

In [29]:
def change_path(pathname_root):
    pathname = pathname_root+bnum+'/'+tnum
    os.chdir(pathname)

In [30]:
def get_nonlin_data():
    os.chdir(recgli_path_root+"/"+bnum+"/"+tnum+'/perf_biopsy')
    nonlin_curve_files = glob.glob('*ave_curve_nonlin*')
    nonlin_curve_df = pd.DataFrame()
    for nonlin_curve_file in nonlin_curve_files: 
        nonlin_curve_data = pd.read_table(nonlin_curve_file, header = None)
        ## we want to extract the exact vial ID as well as the nonlin 
        ## data of interest. 
        vial_name = nonlin_curve_file.split('_')[1]
        nonlin_curve_data = nonlin_curve_data.transpose()
        nonlin_curve_data.columns = nonlin_curve_data.iloc[0]    
        ## ok now we want to create a dataframe from this data that has vialID info and the rest of the information: 
        nonlin_data_vial = {'bnum': bnum, 
                            'tnum': tnum, 
                            'vialid': vial_name, 
                            'phn_nlin': nonlin_curve_data['phn '][1], 
                            'cbvn_nlin': nonlin_curve_data['cbvn '][1], 
                            'recov_nlin': nonlin_curve_data['recov '][1]}
        nonlin_curve_df = nonlin_curve_df.append(nonlin_data_vial, ignore_index = True)
    return nonlin_curve_df

In [31]:
def get_nonpar_data():
    os.chdir(recgli_path_root+"/"+bnum+"/"+tnum+'/perf_biopsy')
    nonpar_curve_files = glob.glob('*ave_curve_nonparam*')
    nonpar_curve_df = pd.DataFrame()
    for nonpar_curve_file in nonpar_curve_files: 
        nonpar_curve_data = pd.read_table(nonpar_curve_file, header = None)
        ## we want to extract the exact vial ID as well as the nonpar 
        ## data of interest. 
        vial_name = nonpar_curve_file.split('_')[1]
        nonpar_curve_data = nonpar_curve_data.transpose()
        nonpar_curve_data.columns = nonpar_curve_data.iloc[0]    
        ## ok now we want to create a dataframe from this data that has vialID info and the rest of the information: 
        nonpar_data_vial = {'bnum': bnum, 
                            'tnum': tnum, 
                            'vialid': vial_name, 
                            'phn_npar': nonpar_curve_data['phn '][1],  
                            'recov_npar': nonpar_curve_data['recov '][1], 
                            'recovn_npar': nonpar_curve_data['recovn '][1] 
                           }
        nonpar_curve_df = nonpar_curve_df.append(nonpar_data_vial, ignore_index = True)
    return nonpar_curve_df

In [32]:
# /home/sf673542/DataWrangling/GetMergeData/Oct2018/ajnr_bnum_tnum.95.csv
bnum_tnum_csv = "/home/sf673542/DataWrangling/GetMergeData/Oct2018/ajnr_bnum_tnum.95.csv"

In [33]:
parser = argparse.ArgumentParser(description='Create a systematic logger of whether biopsies are valid or not through visual inspection')
parser.add_argument("--csv_name",        required=True,    help='Precise name of the csv file that contains the perfusion files of interest.')
parser.add_argument("--csv_dir",         required=True,    help='Precise path of the csv file that contains the perfusion files of interest.')
parser.add_argument("--cohort_name",     required=True,    help='Precise cohort name of the scans of interest (e.g. "po1_preop_recur" or "REC_HGG")')
parser.add_argument("--output_file",     required=False,   help="Name of the output file csv", default = "perf_valid_file.csv")
parser.add_argument("--output_dir",      required=True,    help="Path where output files get written ")
parser.add_argument("-v", "--verbose",                     help = "verbose output", action='store_true', default=False,   required=False)
   


_StoreTrueAction(option_strings=['-v', '--verbose'], dest='verbose', nargs=0, const=True, default=False, type=None, choices=None, help='verbose output', metavar=None)

In [34]:
args = parser.parse_args("--csv_dir /home/sf673542/DataWrangling/GetMergeData/Oct2018/ --csv_name ajnr_bnum_tnum.95.csv --cohort_name REC_HGG --output_file REC_HGG_perf_biopsies.csv --output_dir /home/sf673542/DataWrangling/GetMergeData/Oct2018/".split())


In [35]:
cohort_name     = args.cohort_name
csv_name        = args.csv_name
csv_dir         = args.csv_dir
output_file     = args.output_file
output_dir      = args.output_dir

In [36]:
print("===============================================")
print("scan list dir:     ", csv_dir)
print("scan list name:    ", csv_name)
print("cohort name:       ", cohort_name) 
print("output file name:  ", output_file) 
print("output dir:        ", output_dir)
print("===============================================")


scan list dir:      /home/sf673542/DataWrangling/GetMergeData/Oct2018/
scan list name:     ajnr_bnum_tnum.95.csv
cohort name:        REC_HGG
output file name:   REC_HGG_perf_biopsies.csv
output dir:         /home/sf673542/DataWrangling/GetMergeData/Oct2018/


In [37]:
#   Write cmd to Logfile 
cmd_string = ' '.join(sys.argv)
with open("Logfile", "a") as logfile:
    logfile.write( cmd_string )
    logfile.write( '\n' )

In [38]:
bnum_tnum_df = getting_bnum_tnum_list(csv_name)

In [39]:
if cohort_name == 'po1_preop_recur': 
    recgli_path_root = '/data/RECglioma/archived/'
elif cohort_name == 'REC_HGG': 
    recgli_path_root = '/data/RECglioma/'
else: 
    print('Please use a valid cohort name, REC_HGG or po1_preop_recur.')
    exit(1)

In [40]:
nonlin_total_df = pd.DataFrame()
nonpar_total_df = pd.DataFrame()

In [41]:
for index, row in bnum_tnum_df.iterrows():
    bnum = row['bnum']
    tnum = row['tnum']
    
    print("------------------------------")
    print(bnum)
    print(tnum)

    #####################################
    #   Change into the correct directory
    #####################################
    os.chdir(recgli_path_root+"/"+bnum+"/"+tnum)

    #####################################
    #   Gather the vialIDs 
    #####################################
    tnum_dirs = os.listdir()
    if "roi_analysis" in tnum_dirs: 
        os.chdir('roi_analysis')
        vial_files = glob.glob('*_t1ca_*.idf')
    else: 
        vial_files = []

    #####################################
    #   Change back into the correct directory
    #####################################
    os.chdir(recgli_path_root+"/"+bnum+"/"+tnum)

    #####################################
    #   Gather the data if available: 
    #####################################

    ## If there exists a perf_biopsy folder: 
    if len(glob.glob('perf_biopsy'))>0: 
        
        print('perf_biopsy folder found')
        print('There should be '+str(len(vial_files))+' biopsies for this scan.')
        

        ## Set the perf_biopsy_status to 1 so that we know that it exists
        perf_biopsy_status = 1

        ## If there exists biopsy masks in roi_analysis: 
        if len(vial_files)>0: 
            
            print('vialIDs found in roi_analysis')
            

            ## Set the vial_file_status = 1 so that we know they exist 
            vial_files_status = 1

            ## Instanstiate the nonlin_curve_df and nonpar_curve_df dataframes
            nonlin_curve_df = pd.DataFrame()
            nonpar_curve_df = pd.DataFrame()

            ## Get the nonlin_curve_df for this bnum/tnum
            nonlin_data = get_nonlin_data()
            nonpar_data = get_nonpar_data()

            ## Add these data to the overall dataFrame: 
            nonlin_total_df = nonlin_total_df.append(nonlin_data, ignore_index= True)
            nonpar_total_df = nonpar_total_df.append(nonpar_data, ignore_index= True)

        else: 
            print('----------------------------')
            print('No vial IDs found! No biopsies or run biopsy_make_masks')
            print('----------------------------')

            ## Set the vial_file_status = 1 so that we know they exist 
            vial_files_status = 0

    else: 
        print("------------------------------------------")
        print('No perf_biopsy folder found!! Run perf_biopsy.')
        print('------------------------------------------')

        perf_biopsy_status = 1 

------------------------------
b1338
t8773
perf_biopsy folder found
There should be 3 biopsies for this scan.
vialIDs found in roi_analysis
------------------------------
b2947
t8775
perf_biopsy folder found
There should be 4 biopsies for this scan.
vialIDs found in roi_analysis
------------------------------
b3472
t8783
perf_biopsy folder found
There should be 2 biopsies for this scan.
vialIDs found in roi_analysis
------------------------------
b3499
t8869
perf_biopsy folder found
There should be 2 biopsies for this scan.
vialIDs found in roi_analysis
------------------------------
b3125
t8878
perf_biopsy folder found
There should be 4 biopsies for this scan.
vialIDs found in roi_analysis
------------------------------
b3526
t8943
perf_biopsy folder found
There should be 4 biopsies for this scan.
vialIDs found in roi_analysis
------------------------------
b3527
t8944
perf_biopsy folder found
There should be 2 biopsies for this scan.
vialIDs found in roi_analysis
--------------------

In [42]:
nonlin_total_df

,bnum,cbvn_nlin,phn_nlin,recov_nlin,tnum,vialid
0,b1338,1.38,1.21,100.00,t8773,61B60
1,b1338,1.38,1.02,100.00,t8773,62B60
2,b1338,1.80,1.29,95.57,t8773,63B60
3,b2947,4.88,3.70,88.72,t8775,1B61
4,b2947,3.82,3.03,100.00,t8775,2B61
5,b2947,2.57,1.76,88.01,t8775,3B61
6,b2947,2.17,2.14,81.89,t8775,4B61
7,b3472,0.98,1.07,99.97,t8783,31B61
8,b3472,0.94,0.90,95.56,t8783,33B61
9,b3499,1.68,1.05,100.00,t8869,65B60


In [43]:
nonlin_total_df.index = nonlin_total_df['vialid']
nonpar_total_df.index = nonpar_total_df['vialid']


In [44]:
total_df = nonlin_total_df.join(nonpar_total_df, lsuffix = "_npar")

In [45]:
print(total_df.columns)

Index(['bnum_npar', 'cbvn_nlin', 'phn_nlin', 'recov_nlin', 'tnum_npar',
       'vialid_npar', 'bnum', 'phn_npar', 'recov_npar', 'recovn_npar', 'tnum',
       'vialid'],
      dtype='object')


In [46]:
cols = ['bnum', 'tnum', 'vialid', 'phn_nlin', 'cbvn_nlin', 'recov_nlin', 
                   'phn_npar', 'recov_npar', 'recovn_npar']

In [47]:
total_df = total_df[cols]

In [48]:
total_df = total_df.reset_index(drop = True)

In [49]:
total_df

,bnum,tnum,vialid,phn_nlin,cbvn_nlin,recov_nlin,phn_npar,recov_npar,recovn_npar
0,b1338,t8773,61B60,1.21,1.38,100.00,1.20,100.00,100.00
1,b1338,t8773,62B60,1.02,1.38,100.00,1.10,100.00,100.00
2,b1338,t8773,63B60,1.29,1.80,95.57,1.43,100.00,100.00
3,b2947,t8775,1B61,3.70,4.88,88.72,3.62,93.68,100.00
4,b2947,t8775,2B61,3.03,3.82,100.00,2.77,100.00,100.00
5,b2947,t8775,3B61,1.76,2.57,88.01,1.83,95.04,100.00
6,b2947,t8775,4B61,2.14,2.17,81.89,2.10,87.26,100.00
7,b3472,t8783,31B61,1.07,0.98,99.97,0.99,100.00,100.00
8,b3472,t8783,33B61,0.90,0.94,95.56,0.94,100.00,100.00
9,b3499,t8869,65B60,1.05,1.68,100.00,1.08,100.00,100.00


In [50]:
out_file = output_dir+output_file
out_file

'/home/sf673542/DataWrangling/GetMergeData/Oct2018/REC_HGG_perf_biopsies.csv'

In [51]:
total_df.to_csv(out_file, index = False)

In [52]:
os.listdir("/home/sf673542/DataWrangling/GetMergeData/Oct2018/")

['ajnr_bnum_tnum.95.csv', 'Logfile', 'REC_HGG_perf_biopsies.csv']